In [20]:
import pandas as pd
from sklearn.model_selection import RandomizedSearchCV
from imblearn.over_sampling import ADASYN, SMOTE
from imblearn.under_sampling import RandomUnderSampler
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.ensemble import RandomForestClassifier

In [21]:
df = pd.read_csv('./data_root/clean_dataset.csv')

In [22]:
X = df.drop(['Country','Heart Attack Risk'],axis=1)
Y = df['Heart Attack Risk']

In [23]:
x_train,x_test,y_train,y_test = train_test_split(X,Y,test_size=0.2,random_state=42)

In [24]:
model = RandomForestClassifier()

model.fit(x_train, y_train)

RandomForestClassifier()

In [25]:
train_pred = model.predict(x_train)
test_pred = model.predict(x_test)

In [26]:
print("Treinamento:")
print(confusion_matrix(y_train, train_pred))
print("Teste:")
print(confusion_matrix(y_test, test_pred))

Treinamento:
[[4499    0]
 [   0 2511]]
Teste:
[[1096   29]
 [ 615   13]]


In [27]:
print(classification_report(y_test, test_pred, zero_division=1))
print(classification_report(y_train, train_pred, zero_division=1))

              precision    recall  f1-score   support

           0       0.64      0.97      0.77      1125
           1       0.31      0.02      0.04       628

    accuracy                           0.63      1753
   macro avg       0.48      0.50      0.41      1753
weighted avg       0.52      0.63      0.51      1753

              precision    recall  f1-score   support

           0       1.00      1.00      1.00      4499
           1       1.00      1.00      1.00      2511

    accuracy                           1.00      7010
   macro avg       1.00      1.00      1.00      7010
weighted avg       1.00      1.00      1.00      7010



## Observações do modelo inicial
Podemos observar que nosso modelo com dados normalizados e tratados sem hiperparâmetros teve uma acurâcia de 64% e recall baixo principalmente para risco de ataque cardíaco 1. Mostrando que temos poucas amostras para risco de ataque cardíaco e muitos de pessoas que não apresentaram essa condição. 

In [28]:
Y.value_counts()

Heart Attack Risk
0    5624
1    3139
Name: count, dtype: int64

### Random forest com undersample

In [29]:
x_train,x_test,y_train,y_test = train_test_split(X,Y,test_size=0.2,random_state=42)

In [30]:
rus = RandomUnderSampler(random_state=42)
x_train_resampled, y_train_resampled = rus.fit_resample(x_train, y_train)

print("Distribuição após undersampling:", y_train_resampled.value_counts())

Distribuição após undersampling: Heart Attack Risk
0    2511
1    2511
Name: count, dtype: int64


In [31]:
param_grid = {
    'n_estimators': [100, 200, 300],
    'max_depth': [None, 10, 20],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4]
}

grid_search = GridSearchCV(
    estimator=model, param_grid=param_grid, cv=3, scoring='precision', n_jobs=-1)

grid_search.fit(x_train_resampled, y_train_resampled)
print(grid_search.best_params_)

{'max_depth': 10, 'min_samples_leaf': 2, 'min_samples_split': 5, 'n_estimators': 300}


In [32]:
model = RandomForestClassifier(max_depth=None, min_samples_leaf=1, min_samples_split=2, n_estimators=100)

model.fit(x_train_resampled, y_train_resampled)

RandomForestClassifier()

In [33]:
train_pred_under = model.predict(x_train)
test_pred_under = model.predict(x_test)

In [34]:
print(classification_report(y_test, test_pred_under, zero_division=1))
# print(classification_report(y_train, train_pred, zero_division=1))

              precision    recall  f1-score   support

           0       0.63      0.53      0.58      1125
           1       0.35      0.44      0.39       628

    accuracy                           0.50      1753
   macro avg       0.49      0.49      0.48      1753
weighted avg       0.53      0.50      0.51      1753



### Random forest com oversample

In [36]:
adasyn=ADASYN(sampling_strategy='minority',random_state = 42)
x_train_resampled_over, y_train_resampled_over = adasyn.fit_resample(x_train,y_train)

print("Distribuição após oversample:", y_train_resampled_over.value_counts())

Distribuição após oversample: Heart Attack Risk
0    4499
1    4383
Name: count, dtype: int64


In [38]:
model2 = RandomForestClassifier(n_estimators=300, max_depth=None,min_samples_leaf =1,min_samples_split=2, random_state=42)

model2.fit(x_train_resampled_over, y_train_resampled_over)

RandomForestClassifier(n_estimators=300, random_state=42)

In [39]:
param_grid = {
    'n_estimators': [100, 200, 300],
    'max_depth': [10, 20, None],
    'min_samples_split': [2, 5],
    'min_samples_leaf': [1, 2]
}

grid_search = GridSearchCV(
    estimator=model2, param_grid=param_grid, cv=3, scoring='recall', n_jobs=-1)

grid_search.fit(x_train_resampled_over, y_train_resampled_over)
print(grid_search.best_params_)

{'max_depth': 10, 'min_samples_leaf': 1, 'min_samples_split': 2, 'n_estimators': 100}


In [40]:
train_pred_over = model2.predict(x_train)
test_pred_over = model2.predict(x_test)

print(classification_report(y_test, test_pred_over, zero_division=1))

              precision    recall  f1-score   support

           0       0.63      0.78      0.70      1125
           1       0.33      0.19      0.24       628

    accuracy                           0.57      1753
   macro avg       0.48      0.49      0.47      1753
weighted avg       0.53      0.57      0.54      1753



### Balanceando dataset e usando oversample

In [41]:
x_train_balanced, X_test_balanced, y_train_balanced, y_test_balanced = train_test_split(
    X, Y, test_size=0.2, random_state=42, stratify=Y)

In [42]:
param_grid = {
    'n_estimators': [100, 200, 300],
    'max_depth': [10, 20, None],
    'min_samples_split': [2, 5],
    'min_samples_leaf': [1, 2]
}

grid_search = GridSearchCV(
    estimator=model2, param_grid=param_grid, cv=3, scoring='recall', n_jobs=-1)

grid_search.fit(x_train_balanced, y_train_balanced)
print(grid_search.best_params_)

{'max_depth': None, 'min_samples_leaf': 1, 'min_samples_split': 2, 'n_estimators': 100}


In [43]:
y_pred_balanced = model2.predict(X_test_balanced)

# Avaliação
print(classification_report(y_test_balanced, y_pred_balanced))

              precision    recall  f1-score   support

           0       0.92      0.96      0.94      1125
           1       0.92      0.84      0.88       628

    accuracy                           0.92      1753
   macro avg       0.92      0.90      0.91      1753
weighted avg       0.92      0.92      0.92      1753



### Usando undersample já balanceado

In [44]:
y_pred_balanced_under = model.predict(X_test_balanced)

# Avaliação
print(classification_report(y_test_balanced, y_pred_balanced_under))

              precision    recall  f1-score   support

           0       0.92      0.74      0.82      1125
           1       0.66      0.89      0.75       628

    accuracy                           0.79      1753
   macro avg       0.79      0.81      0.79      1753
weighted avg       0.83      0.79      0.80      1753



## Uso dos Hiperparâmetros e Modelo
Foi usado RandomForest pois existem muitas colunas condicionais com 0 e 1, sendo mais propicio usar arvores de decição aleatorias. Essas colunas com valores binários (0 ou 1) podem se destacar, pois o modelo pode identificar quais dessas variáveis têm maior impacto na classificação também.

Foram testados dois modelos de **RandomForest** para encontrar o melhor desempenho: um utilizando **oversampling** (aumento de dados da classe minoritária) e outro com **undersampling** (remoção de dados da classe majoritária). O melhor modelo foi o de **oversampling**, com dados balanceados utilizao **strafy**.

#### Modelo com Oversample (Melhor Modelo Encontrado):
**Melhores Parâmetros:**
- `'max_depth': None`
- `'min_samples_leaf': 1`
- `'min_samples_split': 2`
- `'n_estimators': 100`

**Explicação dos Parâmetros:**
- **max_depth**: **None** significa que não há limite para a profundidade das árvores, permitindo que elas cresçam sem restrições, o que é vantajoso quando o modelo precisa capturar padrões complexos e não deseja ser restrito a um limite máximo de profundidade.
- **min_samples_leaf**: **1**, assim como no modelo anterior, permite que a árvore seja criada com o menor número possível de amostras, o que pode ser útil quando se tem muitos dados.
- **min_samples_split**: **2**, valor padrão, permitindo a criação de divisões mesmo com poucas amostras.
- **n_estimators**: **100**, número de árvores no modelo, que oferece um bom equilíbrio entre performance e tempo de computação.

**Métricas:**
- **Recall**: 90%
- **Precisão**: 92%
- **Acurácia**: 92%

#### Modelo com Undersample:
**Melhores Parâmetros:**
- `'max_depth': 20`
- `'min_samples_leaf': 1`
- `'min_samples_split': 2`
- `'n_estimators': 100`

**Explicação dos Parâmetros:**
- **max_depth**:Um valor de **20** foi escolhido para evitar overfitting, pois árvores muito profundas podem se ajustar excessivamente aos dados de treino e comprometer a capacidade de generalização do modelo.
- **min_samples_leaf**: O valor **1** foi usado para permitir a criação de folhas com o menor número de amostras possível, o que pode ser útil em alguns casos para capturar padrões mais sutis.
- **min_samples_split**: O valor **2** foi escolhido para permitir divisões mais frequentes, o que pode gerar árvores mais complexas.
- **n_estimators**: **100** foi escolhido, pois é um valor que oferece um bom equilíbrio entre precisão e tempo de treinamento.

**Métricas:**
- **Recall**: 81%
- **Precisão**: 80%
- **Acurácia**: 79%

